In [8]:
import datetime
import pandas as pd
import exifread
import os
from datetime import timedelta
import shutil
from concurrent.futures import ThreadPoolExecutor
import math
import pyfastcopy

def list_files_in_directory(directory):
    file_paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg','.JPG')):
                file_path = os.path.join(root, file)
                file_paths.append(file_path)
            else:
                continue
    return file_paths


def convert_datetime(dt):
    try:
        dt = pd.to_datetime(dt, format='%Y-%m-%d %H-%M-%S')
        return dt.strftime('%Y%m%d_%H%M%S')
    except pd.errors.OutOfBoundsDatetime:
        return 'OutOfBoundsDatetime'

def clean_path(path):
    return os.path.normpath(path)

def process_image(image_path):
    with open(image_path, 'rb') as image_file:
        tags = exifread.process_file(image_file, details=False)
        
        directory = os.path.dirname(image_path)
        filename = os.path.basename(image_path)
        filetype_extension = os.path.splitext(filename)[1]
        make = tags.get('Image Make', 'N/A')
        model = tags.get('Image Model', 'N/A')
        datetime_original = tags.get('EXIF DateTimeOriginal', 'N/A')
        
        return {
            'SourceFile': image_path,
            'Directory': directory,
            'FileName': filename,
            'FileTypeExtension': filetype_extension,
            'Make': make,
            'Model': model,
            'DateTimeOriginal': datetime_original
        }

def read_exif_effort(image_dir):
    file_paths = []
    files = os.listdir(image_dir)
    for file in files:
        if file.lower().endswith(('.jpg', '.jpeg')):
            file_path = os.path.join(image_dir, file)
            file_paths.append(file_path)
    print(len(file_paths))
    with ThreadPoolExecutor() as executor:  # Adjust max_workers as needed
        image_metadata_list = list(executor.map(process_image, file_paths))
    exif_info = pd.DataFrame(image_metadata_list)
    exif_info['Station'] = Station
    exif_info['Camera'] = Camera
    exif_info['DateTimeOriginal'] = pd.to_datetime(exif_info['DateTimeOriginal'], format='%Y:%m:%d %H:%M:%S')
    exif_info['FormattedDateTime'] = exif_info['DateTimeOriginal'].apply(convert_datetime)
    exif_info = exif_info.sort_values(by=['Station', 'Camera', 'DateTimeOriginal']).reset_index(drop=True)
    exif_info['diff'] = exif_info.groupby(['Station', 'Camera'])['DateTimeOriginal'].diff()
    exif_info['diff_sec'] = exif_info["diff"].dt.total_seconds()
    return exif_info

def read_exif(image_dir):
    file_paths = list_files_in_directory(image_dir)
    print(len(file_paths))
    with ThreadPoolExecutor() as executor:  # Adjust max_workers as needed
        image_metadata_list = list(executor.map(process_image, file_paths))
    exif_info = pd.DataFrame(image_metadata_list)
    exif_info['Station'] = Station
    exif_info['Camera'] = Camera
    exif_info['DateTimeOriginal'] = pd.to_datetime(exif_info['DateTimeOriginal'], format='%Y:%m:%d %H:%M:%S')
    exif_info['FormattedDateTime'] = exif_info['DateTimeOriginal'].apply(convert_datetime)
    exif_info = exif_info.sort_values(by=['Station', 'Camera', 'DateTimeOriginal']).reset_index(drop=True)
    exif_info['diff'] = exif_info.groupby(['Station', 'Camera'])['DateTimeOriginal'].diff()
    exif_info['diff_sec'] = exif_info["diff"].dt.total_seconds()
    return exif_info

In [9]:
camera_dir = r"I:\CameraTraps\Guzzler_data\2023_2\SudasariACD\C2"
Station = camera_dir.split("\\")[-2]
print(Station)
Camera = camera_dir.split("\\")[-1]
print(Camera)

SudasariACD
C2


In [10]:
os.listdir(camera_dir)

['20230906_084649__to__20230919_103748',
 '20230919_103806__to__20230921_085623']

In [11]:
exif_df= pd.DataFrame()
###Create Renaming Table
for d in os.listdir(camera_dir):
    start = datetime.datetime.now()
    dir = os.path.join(camera_dir, d)
    exif = read_exif_effort(dir)
    end = datetime.datetime.now()
    print(end - start)
    exif_df=pd.concat([exif_df,exif])

10000
0:00:08.426160
3155
0:00:02.906650


In [12]:
exif_df

,SourceFile,Directory,FileName,FileTypeExtension,Make,Model,DateTimeOriginal,Station,Camera,FormattedDateTime,diff,diff_sec
0,I:\CameraTraps\Guzzler_data\2023_2\SudasariACD...,I:\CameraTraps\Guzzler_data\2023_2\SudasariACD...,SudasariACD_C2_20230906_084649(1).JPG,.JPG,CUDDEBACK,C1,2023-09-06 08:46:49,SudasariACD,C2,20230906_084649,NaT,NaN
1,I:\CameraTraps\Guzzler_data\2023_2\SudasariACD...,I:\CameraTraps\Guzzler_data\2023_2\SudasariACD...,SudasariACD_C2_20230906_084649(2).JPG,.JPG,CUDDEBACK,C1,2023-09-06 08:46:49,SudasariACD,C2,20230906_084649,0 days 00:00:00,0.0
2,I:\CameraTraps\Guzzler_data\2023_2\SudasariACD...,I:\CameraTraps\Guzzler_data\2023_2\SudasariACD...,SudasariACD_C2_20230906_084649(3).JPG,.JPG,CUDDEBACK,C1,2023-09-06 08:46:49,SudasariACD,C2,20230906_084649,0 days 00:00:00,0.0
3,I:\CameraTraps\Guzzler_data\2023_2\SudasariACD...,I:\CameraTraps\Guzzler_data\2023_2\SudasariACD...,SudasariACD_C2_20230906_084733(1).JPG,.JPG,CUDDEBACK,C1,2023-09-06 08:47:33,SudasariACD,C2,20230906_084733,0 days 00:00:44,44.0
4,I:\CameraTraps\Guzzler_data\2023_2\SudasariACD...,I:\CameraTraps\Guzzler_data\2023_2\SudasariACD...,SudasariACD_C2_20230906_084733(2).JPG,.JPG,CUDDEBACK,C1,2023-09-06 08:47:33,SudasariACD,C2,20230906_084733,0 days 00:00:00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3150,I:\CameraTraps\Guzzler_data\2023_2\SudasariACD...,I:\CameraTraps\Guzzler_data\2023_2\SudasariACD...,SudasariACD_C2_20230921_085614(2).JPG,.JPG,CUDDEBACK,C1,2023-09-21 08:56:14,SudasariACD,C2,20230921_085614,0 days 00:00:00,0.0
3151,I:\CameraTraps\Guzzler_data\2023_2\SudasariACD...,I:\CameraTraps\Guzzler_data\2023_2\SudasariACD...,SudasariACD_C2_20230921_085614(3).JPG,.JPG,CUDDEBACK,C1,2023-09-21 08:56:14,SudasariACD,C2,20230921_085614,0 days 00:00:00,0.0
3152,I:\CameraTraps\Guzzler_data\2023_2\SudasariACD...,I:\CameraTraps\Guzzler_data\2023_2\SudasariACD...,SudasariACD_C2_20230921_085623(1).JPG,.JPG,CUDDEBACK,C1,2023-09-21 08:56:23,SudasariACD,C2,20230921_085623,0 days 00:00:09,9.0
3153,I:\CameraTraps\Guzzler_data\2023_2\SudasariACD...,I:\CameraTraps\Guzzler_data\2023_2\SudasariACD...,SudasariACD_C2_20230921_085623(2).JPG,.JPG,CUDDEBACK,C1,2023-09-21 08:56:23,SudasariACD,C2,20230921_085623,0 days 00:00:00,0.0


In [13]:
exif_df.to_csv(os.path.join(camera_dir,f"{Station}_{Camera}_exif_summary.csv"))